In [2]:
import numpy as np
import torch

In [9]:
r = np.load('32_result.npy')
for i in range(6):
    print((r==i).sum())

2547
52
38
127
53
183


In [7]:
for i in range(6):
    print((r==i).sum())

2610
203
116
27
0
44


In [187]:
def cal_mIoU(pred, true, cal_cls=[0,1],device='cpu'):
    """
    pred: size=(batch,N)
    true: size=(batch,N)
    cal_cls: [0,1,2,3,4,...]
    """

    num_cls = len(cal_cls)
    if not isinstance(pred,torch.Tensor):
        pred = torch.tensor(pred).to(device)
    if not isinstance(true,torch.Tensor):
        true = torch.tensor(true).to(device)

    iou = torch.zeros((num_cls)).to(device)
    for i in range(num_cls):
        a, b = pred==cal_cls[i], true==cal_cls[i]
        iou[i] = (a & b).sum() / ((a | b).sum() + 1e-7) 
    mIoU = iou.mean()
    return mIoU,iou

In [210]:
p,t = [],[]
for i in range(20):
    pred = np.random.randint(0,6,(10,3000))
    true = np.random.randint(0,6,(10,3000))
    mIoU, iou_batch = cal_mIoU(pred,true,cal_cls=[1,2,3,4,5],device='cuda')

In [184]:
mIoU,iou_batch

(tensor(0.0909, device='cuda:0'),
 tensor([0.0909, 0.0910, 0.0910, 0.0909, 0.0909], device='cuda:0'))

In [195]:
pred = torch.rand([1, 6, 3000])

In [203]:
pred.max(1)[1].shape

torch.Size([1, 3000])

In [227]:
pc = torch.rand((10,3,3000))
mask = torch.Tensor(pred).unsqueeze(1)==1

In [249]:
pred.shape

(10, 3000)

In [229]:
mask*pc

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.2200, 0.0000, 0.3323],
         [0.0000, 0.0000, 0.0000,  ..., 0.7341, 0.0000, 0.2202],
         [0.0000, 0.0000, 0.0000,  ..., 0.4318, 0.0000, 0.9321]],

        [[0.0000, 0.9469, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.4586, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1418, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1468, 0.0602],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5488, 0.5393],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5953, 0.3141]],

        ...,

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

 

In [251]:
mask.sum(dim=2)

tensor([[498],
        [501],
        [518],
        [490],
        [482],
        [494],
        [477],
        [521],
        [486],
        [526]])

In [252]:
mean_xyz = (mask*pc).sum(-1)/mask.sum(dim=2)
mean_xyz

tensor([[0.5269, 0.5143, 0.5058],
        [0.4988, 0.5079, 0.4959],
        [0.4909, 0.5151, 0.4845],
        [0.5121, 0.4927, 0.4964],
        [0.5051, 0.5056, 0.4916],
        [0.4858, 0.4957, 0.4980],
        [0.5086, 0.4800, 0.5029],
        [0.5048, 0.4955, 0.4860],
        [0.5020, 0.4946, 0.4940],
        [0.5078, 0.5058, 0.4788]])

In [253]:
var_temp = ((mask*pc - mean_xyz.unsqueeze(-1))**2).sum(1)*mask.squeeze(1)
(var_temp).sum(1)/mask.sum(dim=2).squeeze(1)

tensor([0.2498, 0.2452, 0.2463, 0.2470, 0.2477, 0.2530, 0.2463, 0.2542, 0.2469,
        0.2485])

In [3]:
a = torch.tensor([0.3213, 0.0608, 0.0665, 0.0730, 0.0813, 0.1210])
b = torch.tensor([0.3370, 0.0320, 0.0433, 0.0593, 0.0749, 0.0886])

In [6]:
a = torch.tensor([0.3224, 0.0446, 0.0361, 0.0276, 0.0217, 0.0470])
b = torch.tensor([0.3383, 0.0169, 0.0131, 0.0141, 0.0146, 0.0139])

In [7]:
(torch.abs(a - b) / b)*100

tensor([  4.7000, 163.9053, 175.5725,  95.7447,  48.6301, 238.1295])